This problem is pretty damn hard without the proper theoretical knowledge, but easy-ish with it. The key is <https://en.wikipedia.org/wiki/Continued_fraction#Best_rational_approximations>. I quote:

> One can choose to define a best rational approximation to a real number $x$ as a rational number $n/d$, $d > 0$, that is closer to $x$ than any approximation with a smaller or equal denominator. The simple continued fraction for $x$ can be used to generate all of the best rational approximations for $x$ by applying these three rules:
>
> - Truncate the continued fraction, and reduce its last term by a chosen amount (possibly zero).
> - The reduced term cannot have less than half its original value.
> - If the final term is even, half its value is admissible only if the corresponding semiconvergent is better than the previous convergent.

So we just need to calculate continued fraction terms, stop and check semiconvergents once the denominator blows up.

Sage has a `nearby_rational` method that solves this problem (we do need to take care to generate enough decimal places beforehand) so we can cheat and run with it (it's slow, takes >20s), or just use it for crosscheck.

To calculate continued fractions quickly ourselves, let

$$[a_0; a_1, \dots, a_{n-1}, x] = \frac{\alpha_n x + \beta_n}{y_n x + \delta_n}$$

Then it's easy to show

$$(\alpha_0,\, \beta_0,\, \gamma_0,\, \delta_0) = (1,\, 0,\, 0,\, 1),$$
$$(\alpha_{n+1},\, \beta_{n+1},\, \gamma_{n+1},\, \delta_{n+1}) = (\alpha_n a_n + \beta_n,\, \alpha_n,\, \gamma_n a_n + \delta_n,\, \gamma_n).$$

Therefore the $n$-th convergent is simply $\frac{\alpha_{n+1}}{\beta_{n+1}}$. To compute $a_n$, notice that it is the floor of $x$ s.t. 
$[a_0; a_1, \dots, a_{n-1}, x] = \sqrt{k}$, so we can solve for this $x$ and derive

$$a_n = \lfloor x \rfloor = \left\lfloor \frac{(\delta_n \alpha_n - \beta_n \gamma_n)\sqrt{k} + (\delta_n \gamma_n k - \beta_n \alpha_n)}{\alpha_n^2 - \gamma_n^2 k} \right\rfloor.$$

The Python implementation eventually took ~2.4s.

In [1]:
#!/usr/bin/env python3

import math
from fractions import Fraction


BOUND = 10 ** 12


def best_approx_denom(n):
    a, b, c, d = 1, 0, 0, 1
    numeric_sqrt = math.sqrt(n)
    if int(numeric_sqrt) ** 2 == n:
        return 0
    while True:
        mm = d * a - b * c
        nn = d * c * n - b * a
        ll = a * a - c * c * n
        term = math.floor((mm * numeric_sqrt + nn) / ll)
        prev_b, prev_d = b, d
        a, b, c, d = a * term + b, a, c * term + d, c
        if c > BOUND:
            if term > 1:
                for t in range(term - 1, int(term / 2), -1):
                    denom = d * t + prev_d
                    if denom <= BOUND:
                        # print(f"{n}\t{b * t + prev_b}/{denom}")
                        return denom
                if term % 2 == 0:
                    t = term // 2
                    denom = d * t + prev_d
                    if denom <= BOUND:
                        # Test which approximation is better,
                        #
                        #   r2 = (b * t + prev_b)/(d * t + prev_d)
                        #
                        # or the previous convergent r1 = b/d.
                        #
                        # r2 is better iff
                        #
                        #   (r1 - sqrt(n))^2 > (r2 - sqrt(n))^2.
                        r2 = Fraction(b * t + prev_b, denom)
                        r1 = Fraction(b, d)
                        if (r1 > r2 and (r1 + r2) ** 2 > 4 * n) or (
                            r1 < r2 and (r1 + r2) ** 2 < 4 * n
                        ):
                            # print(f"{n}\t{r2}")
                            return denom
            # print(f"{n}\t{b}/{d}")
            return d


def main():
    denom_sum = 0
    for n in range(2, 100_001):
        denom_sum += best_approx_denom(n)
    print(denom_sum)


if __name__ == "__main__":
    main()


57060635927998347
